# Just train and save a resnet18 on CIFAR10 with pytorch lightning

In [1]:
%matplotlib inline

In [2]:
import torch
import torchvision
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet18
import torch.nn.functional as F
import torch.nn as nn
from torchvision.datasets import CIFAR10

import pytorch_lightning as pl
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import loggers as pl_loggers

import torchmetrics
from tqdm.notebook import tqdm
import lightly

/home/shatz/miniconda3/envs/morebetter3/lib/python3.9/site-packages/lightly/api/version_checking.py:57: Warning: You are using lightly version 1.1.18. There is a newer version of the package available. For compatability reasons, please upgrade your current version: pip install lightly==1.1.20
  warnings.warn(Warning(warning))


# DATA

In [3]:
# data params
num_workers = 6
batch_size = 512

In [4]:
# The dataset structure should be like this:
# cifar10/train/
#  L airplane/
#    L 10008_airplane.png
#    L ...
#  L automobile/
#  L bird/
#  L cat/
#  L deer/
#  L dog/
#  L frog/
#  L horse/
#  L ship/
#  L truck/
path_to_train = './data/cifar10_lightly/train/'
path_to_test = './data/cifar10_lightly/test/'

In [5]:
# ------------- transforms ------------------- #

# Augmentations typically used to train on cifar-10
train_classifier_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    #     torchvision.transforms.Normalize(
    #         mean=(0.5, 0.5, 0.5),
    #         std=(0.5, 0.5, 0.5),
    #     ),
])

# No additional augmentations for the test set
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((32, 32)),
    torchvision.transforms.ToTensor(),
    #     torchvision.transforms.Normalize(
    #         mean=lightly.data.collate.imagenet_normalize['mean'],
    #         std=lightly.data.collate.imagenet_normalize['std'],
    #     ),
])

# --------------- datasets --------------------- #
dataset_train_classifier = lightly.data.LightlyDataset(
    input_dir=path_to_train,
    transform=train_classifier_transforms
)

dataset_test = lightly.data.LightlyDataset(
    input_dir=path_to_test,
    transform=test_transforms
)

# ------------------ dataloaders ----------------- #
train_dataloader = torch.utils.data.DataLoader(
    dataset_train_classifier,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)

val_dataloader = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

# Model

In [6]:
# model hyperparams
max_epochs = 200

In [7]:
from plr18 import plr18
model = plr18()

# Train

In [8]:
# # Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

Reusing TensorBoard on port 6006 (pid 236117), started 5 days, 21:30:28 ago. (Use '!kill 236117' to kill it.)

In [9]:
# you can also define a checkpoint callback to save best model like keras.
checkpoint_callback = ModelCheckpoint(
    dirpath='./saved_models/resnet_80',
    filename='{epoch}-{val_loss:.2f}-{val_acc:.2f}',
    save_top_k=5,
    verbose=True,
    monitor='avg_val_acc',
    mode='max'
)

In [10]:
tb_logger = pl_loggers.TensorBoardLogger(save_dir='./lightning_logs/', name='resnet_80')
trainer = Trainer(gpus=1, callbacks=[checkpoint_callback], max_epochs=80, logger=tb_logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [11]:
trainer.fit(model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type             | Params
----------------------------------------------
0 | model    | ResNet           | 11.2 M
1 | criteria | CrossEntropyLoss | 0     
----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/shatz/miniconda3/envs/morebetter3/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Epoch 77, global step 7565: avg_val_acc reached 0.79922 (best 0.80195), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_80/epoch=77-val_loss=0.70-val_acc=0.80.ckpt" as top 5


Validating: 0it [00:00, ?it/s]

Epoch 78, global step 7662: avg_val_acc was not in top 5


Validating: 0it [00:00, ?it/s]

Epoch 79, global step 7759: avg_val_acc reached 0.80526 (best 0.80526), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_80/epoch=79-val_loss=0.70-val_acc=0.80.ckpt" as top 5
